In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
from datetime import datetime

In [3]:
data = pd.read_csv('Weather.csv', index_col=0)
data.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1901,17.99,19.43,23.49,26.41,28.28,28.60,27.49,26.98,26.26,25.08,21.73,18.95
1,1902,19.00,20.39,24.10,26.54,28.68,28.44,27.29,27.05,25.95,24.37,21.33,18.78
2,1903,18.32,19.79,22.46,26.03,27.93,28.41,28.04,26.63,26.34,24.57,20.96,18.29
3,1904,17.77,19.39,22.95,26.73,27.83,27.85,26.84,26.73,25.84,24.36,21.07,18.84
4,1905,17.40,17.79,21.78,24.84,28.32,28.69,27.67,27.47,26.29,26.16,22.07,18.71


In [4]:
data.shape

(117, 13)

In [5]:
data1 = pd.melt(data, id_vars='YEAR', value_vars=data.columns[1:])
data1.head()

,YEAR,variable,value
0,1901,JAN,17.99
1,1902,JAN,19.00
2,1903,JAN,18.32
3,1904,JAN,17.77
4,1905,JAN,17.40


In [6]:
data.shape

(117, 13)

Convert String to Datetime

In [7]:
data1['Date'] = data1['variable'] + ' ' + data1['YEAR'].astype(str)
data1.loc[:,'Date'] = data1['Date'].apply(lambda x : datetime.strptime(x, '%b %Y'))
data1.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_17660\1506109968.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data1.loc[:,'Date'] = data1['Date'].apply(lambda x : datetime.strptime(x, '%b %Y'))


,YEAR,variable,value,Date
0,1901,JAN,17.99,1901-01-01
1,1902,JAN,19.00,1902-01-01
2,1903,JAN,18.32,1903-01-01
3,1904,JAN,17.77,1904-01-01
4,1905,JAN,17.40,1905-01-01


Temperature Through Time

In [8]:
data1.columns=['Year', 'Month', 'Temprature', 'Date']
data1.sort_values(by='Date', inplace=True) ## To get the time series right.

fig = go.Figure(layout = go.Layout(yaxis=dict(range=[0, data1['Temprature'].max()+1])))
fig.add_trace(go.Scatter(x=data1['Date'], y=data1['Temprature']), )
fig.update_layout(title='Temprature Throught Timeline:',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.update_layout(xaxis=go.layout.XAxis(
    rangeselector=dict(
        buttons=list([dict(label="Whole View", step="all"),
                      dict(count=1,label="One Year View",step="year",stepmode="todate")                      
                     ])),
        rangeslider=dict(visible=True),type="date")
)
fig.show()

Insight:

1. Mei 1921 menjadi bulan terpanas di India. 

2. Desember, Januari dan februari adalah bulan terdingin (Musim Dingin)

3. April, Mei, Juni, Juli dan Agustus menjadi bulan-bulan terpanas. (Musim Panas)

Tapi, ini bukan musim yang ada di India. India memiliki empat musim utama di India:

1. Musim Dingin : Desember, Januari dan Februari.

2. Musim Panas (Pra Musim Hujan) : Maret, April dan Mei.

3. Monsun : Juni, Juli, Agustus dan September.

4. Musim Gugur (Pasca Musim Monsun) : Oktober dan November.

In [9]:
# data1.columns=['Year', 'Month', 'Temperature', 'Date']
# data1.sort_values(by='Date', inplace=True)

# fig = go.Figure(layout=go.layout(yaxis=dict(range=[0, data1['Temperature'].max()+1])))
# fig.add_trace(go.Scatter(x=data1['Date'], y=data1['Temperature']),)
# fig.update_layout(title='Temperature Through Timeline:',xaxis_title='Time', yaxis_title='Temperature in Degress')

# fig.update_layout(xaxis=go.layout.XAxis(
#     rangeselector=dict(buttons=list([dict(label='Whole View', step='all'),dict(count=1, label='One Year View', step='year', stepmode='todate')])),
#     rangeslider=dict(visible=True), type='date'))
# fig.show()

Warmest / Coldest / Average

In [11]:
fig = px.box(data1, 'Month', 'Temprature')
fig.update_layout(title='Warmest, Coldest and Median Monthly')
fig.show()

Insight:

Januari adalah bulan dengan temperatur paling rendah

Mei temperatur paling tinggi

Juli std paling rendah, artinya suhu di juli tidak begitu banyak variasi

### Clustering Using KMeans

In [12]:
from sklearn.cluster import KMeans

In [14]:
sse = []
target = data1['Temprature'].to_numpy().reshape(-1,1)
n_cluster = list(range(1,10))

In [15]:
for k in n_cluster:
    km = KMeans(n_clusters=k)
    km.fit(target)
    sse.append(km.inertia_)

In [16]:
fig = go.Figure(data=[
    go.Scatter(x = n_cluster, y=sse, mode='lines'),
    go.Scatter(x = n_cluster, y=sse, mode='markers')
])

In [17]:
fig.update_layout(title='Evaluation on Number of Cluster:',
xaxis_title = 'Number of Clusters:',
yaxis_title = 'Sum of Squared Distance',
showlegend=False)

fig.show()

n_cluster =  2 or 3

In [18]:
km = KMeans(3)

km.fit(data1['Temprature'].to_numpy().reshape(-1,1))
data1.loc[:, 'Temp Labels'] = km.labels_

In [19]:
fig = px.scatter(data1, 'Date', 'Temprature', color='Temp Labels')
fig.update_layout(title = 'Temperature Clusters', xaxis_title="Date", yaxis_title="Temprature")

fig.show()

In [20]:
fig = px.histogram(x=data1['Temprature'], nbins=200, histnorm='density')
fig.update_layout(title='Frequency Chart of Temprature Reading',
xaxis_title='Temprature', yaxis_title='Count')

Ada cluster dari 26,2-27,5 dan suhu rata-rata untuk sebagian besar bulan selama sejarah adalah antara 26,8-26,9

Yearly Average Temprature

In [22]:
data['Yearly Mean'] = data.iloc[:,1:].mean(axis=1)

fig = go.Figure(data=[
    go.Scatter(name='Yearly Temprature', x=data['YEAR'], y=data['Yearly Mean'], mode='lines'),
    go.Scatter(name='Yearly Temprature', x=data['YEAR'], y=data['Yearly Mean'], mode='markers'),
])

fig.update_layout(title='Yearly Mean Temprature :',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.show()

Suhu rata-rata tahunan tidak meningkat sampai tahun 1980, baru setelah 1979 ada penungkatan terhadap suhu rata2 tahunan.

Setelah tahun 2015, suhu tahunan meningkat drastis.

### Monthly Temprature Through History

In [23]:
fig = px.line(data1, 'Year', 'Temprature', facet_col='Month', facet_col_wrap=4)
fig.update_layout(title='Monthly temprature throught history:')
fig.show()

### Seasonal Weather Analysis

In [24]:
data['Winter'] = data[['DEC', 'JAN', 'FEB']].mean(axis=1)
data['Summer'] = data[['MAR', 'APR', 'MAY']].mean(axis=1)
data['Monsoon'] = data[['JUN', 'JUL', 'AUG', 'SEP']].mean(axis=1)
data['Autumn'] = data[['OCT', 'NOV']].mean(axis=1)
seasonal_df = data[['YEAR', 'Winter', 'Summer', 'Monsoon', 'Autumn']]
seasonal_df = pd.melt(seasonal_df, id_vars='YEAR', value_vars=seasonal_df.columns[1:])
seasonal_df.columns=['Year', 'Season', 'Temprature']

fig = px.scatter(seasonal_df, 'Year', 'Temprature', facet_col='Season', facet_col_wrap=2, trendline='ols')
fig.update_layout(title='Seasonal mean tempratures throught years:')
fig.show()

In [25]:
px.scatter(data1, 'Month', 'Temprature', size='Temprature', animation_frame='Year')

### Weather Forecasting with Machine Learning

In [26]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [27]:
data2 = data1[['Year', 'Month', 'Temprature']].copy()
data2 = pd.get_dummies(data2)

In [28]:
x = data2.drop(columns='Temprature')
y = data2[['Temprature']]

In [29]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [30]:
model = DecisionTreeRegressor()
model.fit(x_train,y_train)

DecisionTreeRegressor()

In [31]:
y_pred = model.predict(x_test)
r2_score(y_test,y_pred)

0.9632351113933759

In [37]:
import pickle

with open('modelDtree.pkl','wb') as f:
    pickle.dump(model,f)

In [34]:
next_Year = data1[data1['Year']==2017][['Year', 'Month']]
next_Year.Year.replace(2017,2018, inplace=True)
next_Year= pd.get_dummies(next_Year)
temp_2018 = model.predict(next_Year)

In [35]:
temp_2018 = {'Month':data1['Month'].unique(), 'Temprature':temp_2018}
temp_2018=pd.DataFrame(temp_2018)
temp_2018['Year'] = 2018
temp_2018

,Month,Temprature,Year
0,JAN,20.92,2018
1,FEB,23.58,2018
2,MAR,25.58,2018
3,APR,29.17,2018
4,MAY,30.47,2018
5,JUN,28.15,2018
6,JUL,28.18,2018
7,AUG,28.12,2018
8,SEP,27.72,2018
9,OCT,27.24,2018


In [36]:
forecasted_temp = pd.concat([data1,temp_2018], sort=False).groupby(by='Year')['Temprature'].mean().reset_index()
fig = go.Figure(data=[
    go.Scatter(name='Yearly Mean Temprature', x=forecasted_temp['Year'], y=forecasted_temp['Temprature'], mode='lines'),
    go.Scatter(name='Yearly Mean Temprature', x=forecasted_temp ['Year'], y=forecasted_temp['Temprature'], mode='markers')
])
fig.update_layout(title='Forecasted Temprature:',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.show()